<a href="https://colab.research.google.com/github/luansantiago0/ProjetosFacul/blob/master/TranfsFNG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def remove_unit_productions(grammar):
    updated_grammar = {variable: productions for variable, productions in grammar.items()}
    
    # Passo 1: Encontrar produções unitárias e adicioná-las à gramática atualizada
    unit_productions = []
    
    for variable in grammar:
        productions = grammar[variable]
        
        for production in productions:
            if len(production) == 1 and production.isupper():
                unit_productions.append((variable, production))
    
    for unit_production in unit_productions: #Itera sobre cada produção unitária encontrada na gramática.
        variable, unit_variable = unit_production #Desempacota a variável original e a variável unitária da produção unitária atual.
        
        if unit_variable in updated_grammar: # Verifica se a variável unitária está presente na gramática atualizada.
            unit_productions_to_add = [(variable, production) for production in updated_grammar[unit_variable]] #Cria uma lista de produções para adicionar, que consiste em pares de (variável original, produção) encontrados na variável unitária.
            
def remove_empty_productions(grammar):
    updated_grammar = {variable: [production for production in productions if production != ''] for variable, productions in grammar.items()}
    
    nullable_variables = set()
    updated_productions = {variable: set(productions) for variable, productions in updated_grammar.items()}
    
    # Passo 1: Encontrar variáveis nulas
    while True:
        nullable_variables_prev = set(nullable_variables)
        
        for variable, productions in updated_grammar.items():
            if any(all(symbol in nullable_variables for symbol in production) for production in productions):
                nullable_variables.add(variable)
        
        if nullable_variables == nullable_variables_prev:
            break
    
    # Passo 2: Remover produções vazias
    for variable, productions in updated_grammar.items():
        updated_productions[variable] = set(production for production in productions if production != 'ε')
    
    # Passo 3: Adicionar produções adicionais para variáveis nulas
    for variable, productions in updated_grammar.items():
        for nullable_variable in nullable_variables:
            updated_productions[variable].update(production.replace(nullable_variable, '') for production in productions if nullable_variable in production)
    
    return {variable: list(productions) for variable, productions in updated_productions.items()}


def rename_variables(grammar):
    updated_grammar = {variable: [production for production in productions] for variable, productions in grammar.items()}
    
    # Gerar novos nomes de variáveis
    new_variable_counter = 0
    used_variables = set(updated_grammar.keys())
    new_variable_mapping = {}
    
    def generate_new_variable():
        nonlocal new_variable_counter
        
        while True:
            new_variable_counter += 1
            new_variable = 'X' + str(new_variable_counter)
            
            if new_variable not in used_variables:
                used_variables.add(new_variable)
                return new_variable
    
    # Substituir variáveis nas produções
    for variable, productions in updated_grammar.items():
        updated_productions = []
        
        for production in productions:
            new_production = ''
            
            for symbol in production:
                if symbol.isupper() and symbol != variable:
                    if symbol not in new_variable_mapping:
                        new_variable_mapping[symbol] = generate_new_variable()
                    
                    new_production += new_variable_mapping[symbol]
                else:
                    new_production += symbol
            
            updated_productions.append(new_production)
        
        updated_grammar[variable] = updated_productions
    
    return updated_grammar


def remove_unit_productions(grammar):
    updated_grammar = {variable: [production for production in productions] for variable, productions in grammar.items()}
    
    # Passo 1: Encontrar produções unitárias e adicioná-las à gramática atualizada
    unit_productions = []
    
    for variable in grammar:
        productions = grammar[variable]
        
        for production in productions:
            if len(production) == 1 and production.isupper():
                unit_productions.append((variable, production))
    
    for unit_production in unit_productions:
        variable, unit_variable = unit_production
        
        if unit_variable in updated_grammar:
            unit_productions_to_add = [(variable, production) for production in updated_grammar[unit_variable]]
            
            for unit_production_to_add in unit_productions_to_add: # Itera sobre cada produção a ser adicionada.
                if unit_production_to_add not in unit_productions: #  Verifica se a produção a ser adicionada não está presente nas produções unitárias.
                    unit_productions.append(unit_production_to_add) # Adiciona a produção unitária à lista de produções unitárias.
    
    for unit_production in unit_productions:
        variable, unit_variable = unit_production
        
        if unit_variable in updated_grammar:
            updated_productions = updated_grammar[variable]
            unit_productions_to_add = [(variable, production) for production in updated_grammar[unit_variable]]
            
            for unit_production_to_add in unit_productions_to_add: # Itera sobre cada produção a ser adicionada.
                if unit_production_to_add not in updated_productions:
                    updated_productions.append(unit_production_to_add)
    
    # Passo 2: Remover as produções unitárias da gramática atualizada
    for variable in updated_grammar:
        updated_productions = updated_grammar[variable]
        updated_grammar[variable] = [production for production in updated_productions if len(production) > 1 or not production.isupper()]
    
    return updated_grammar


def transform_to_fng(grammar):
    updated_grammar = dict(grammar)
    
    updated_grammar = remove_empty_productions(updated_grammar)
    updated_grammar = remove_unit_productions(updated_grammar)
    updated_grammar = rename_variables(updated_grammar)
    
    return updated_grammar


# Exemplo de gramática
grammar = {
    'S': ['AB', 'CSB'],
    'A': ['aB', 'C'],
    'B': ['bbB', 'b']
}

# Transformar a gramática para FNG
fng_grammar = transform_to_fng(grammar)

# Imprimir a gramática resultante
for variable, productions in fng_grammar.items():
    print(f"{variable} -> {' | '.join(productions)}")



S -> X1X2 | X3SX2
A -> aX2
B -> b | bbB
